In [0]:
from fastai.vision import *

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
ls '/content/gdrive/My Drive/data/cats-dogs/'

In [0]:
path = Path('/content/gdrive/My Drive/data/cats-dogs/')
train_dir = path/'train'
test_dir = path/'test'

## View data

In [0]:
data = ImageDataBunch.from_folder(path, train=train_dir, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224,num_workers=4).normalize(imagenet_stats)

In [0]:
data.classes

In [0]:
data.show_batch(rows=3, figsize=(7,8))

## Train model

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('stage-1')

## Fine Tune Transfer Learning

In [0]:
learn.unfreeze()

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(2, max_lr=slice(3e-5,3e-4))

In [0]:
learn.save('stage-2')

## Interpretation

In [0]:
learn.load('stage-2');

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

## Data Augmentation

In [0]:
tfms = [rotate(degrees=(-20,20)), symmetric_warp(magnitude=(-0.3,0.3))]
data = ImageDataBunch.from_folder(path, train=train_dir, valid_pct=0.2,
        ds_tfms=(tfms, []), size=224,num_workers=4).normalize(imagenet_stats)

In [0]:
fig,axes = plt.subplots(1,4,figsize=(8,2))

ds = data.train_ds
for ax in axes: ds[0][0].apply_tfms(tfms).show(ax=ax)

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [0]:
learn.fit_one_cycle(1)